In [1]:
%matplotlib inline
#%matplotlib notebook
#%matplotlib widget
import matplotlib 
import numpy as np
import pandas as pd
import os, sys
#import ipywidgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection    
 
# use LaTeX, choose nice some looking fonts and tweak some settings
matplotlib.rc('font', family='serif')
matplotlib.rc('font', size=16)
matplotlib.rc('legend', fontsize=16)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('legend', handlelength=1.5)
matplotlib.rc('legend', frameon=True)
matplotlib.rc('xtick.major', pad=7)
matplotlib.rc('xtick', direction="in")
matplotlib.rc('ytick', direction="in")
matplotlib.rc('xtick', top = True)
matplotlib.rc('ytick', right =True )
matplotlib.rc('xtick.minor', pad=7)
matplotlib.rc('text', usetex=True)
# matplotlib.rc('text.latex', 
#               preamble=[r'\usepackage[T1]{fontenc}',
#                         r'\usepackage{amsmath}',
#                         r'\usepackage{txfonts}',
#                         r'\usepackage{textcomp}'])

matplotlib.rc('figure', figsize=(12, 9))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device('cuda:0')

Marcov Process
===

## 조건부 확률

- $\mathbb{P}[A\mid B]$ 는 조건부 확률이다.
- 상태집합 $S$ 는 Process 에서 가능한 모든 상태의 집합을 의미한다. $S=\{S_1,\,S_2,\ldots\,S_N\}$



## Marcov Property and Marcove Process
- 어떤 시간 순서 $t_1,\,t_2,\ldots,\,t_n$ 을 생각하자. $t_i < t_{i+1}$ 이다. 이 때 $s_i$ 는 시간 $t_i$ 에서의 상태라 하자. $\mathbb{P}[s_{i+1}|s_i\cdots s_1]$ 은 $s_1,\ldots,\,s_{i}$ 상태를 겪은 후에 $s_{i+1}$ 상태가 나타날 확률이다. 

- Marcov property 는 $\mathbb{P}[s_{i+1} \mid s_{i}\cdots s_1] = \mathbb{P}[s_{i+1}|s_{i}]$ 인 특성이다. 즉 어떤 상태 $s_{i+1}$ 이 나타날 확률은 바로 직전의 상태 $s_{i}$ 에만 의존하며 그 이전의 history에 무관한 성질이다.

- 마르코프 과정(Markov Process)은 마르코프 가정을 만족하는 연속적인 일련의 상태 $(s_1,s_2,\dots,s_t)$ 와 상태 전이 확률(state transition probability) $\mathcal{P}$ 로 구성된다.



## 상태변이확률 (State Transition Probability)

- 상태변이확률 $\mathcal{P}_{ss'}$ 은 다음과 같이 정의된다.

$$
\mathcal{P}_{ss'}= \mathbb{P}[s'=s_{i+1}\mid s=s_i]\;.
$$

  
## 마코프 보상 프로세스 (Marcov Rewared Process, MRP)

- 보상이란 어떤 상태에서 다음 단계의 상태로 이동할 때 환경으로부터 피드백 받는 스칼라 실수값으로 다음과 같이 표현된다.

$$
\mathcal{R}_{s}= \mathbb{E}[R_{t+1}|s_t=s]\,.
$$

- **감쇄 계수 (discount factor) $\gamma$** : 미래 가치를 현재 기준으로 어느 정도로 반영할 것인지에 대한 factor. $0 \le \gamma \le 1$ 이며 $\gamma=0$ 이면 미래상태에 대한 가치를 전혀 고려 하지 않은 값이고, $\gamma=1$ 이면 모든 미래상태의 가치를 현재와 같은 비중으로 고려한다는 뜻이다.

- **상태가치함수 (state-value function) $v(s)$** : 어떤 상태 $s$ 에서 미래에 발생 할 수 있는 상태의 보상값을 모두 더한 값.

$$
R_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots = \sum_{k=0}^\infty \gamma^k r_{t+k+1}
$$

- $t$-th 단계에서의 상태가치함수를 열벡터로 표시하여 $v^{(t)} = \begin{bmatrix} v^{t}_1\\ \vdots \\ v^{t}_N\end{bmatrix}$ 라 하자. 각 상태에서의 보상을 행렬로 표현하여 $\mathbf{R}^{(t)}$ 이라 하고, 상태변이확률을 행렬로 표현하여 $\mathbf{P}$ 라 하자. $v^{(t)}$ 의 $i$-th row 는 $t$-th stage 에서 $s_i\in S$ 상태일 때의 상태가치 함수이다. 이것은 현재 단계에서의 보상 $R$ 과 $(t+1)$-th 단계에서의 상태가치 벡터 $v^{(t+1)}$, 그리고 상태전이행렬$\mathbf{P}$ 를 사용하여 다음과 같이 표현 할 수 있다.  

$$
v^{(t)}=\mathbf{R}^{(t)}+\gamma \mathbf{P} v^{(t+1)}
$$

- 우리는 어떤 단계에서의 상태가치함수보다는 궁극적으로 최종적인 상태가치함수에 관심이 있으며, 이것이 수렴함을 확신 할 수 있다. 이것을 $v$ 라 하자. 또한 infinite process 를 가정 할 때 $\mathbf{R}^{(t)}$ 가 단계 $t$ 에 의존하지 않는다는것도 알 수 있다. 이것을 $\mathbf{R}$ 이라 하자. 그렇다면 다음을 만족한다.

$$
v=\mathbf{R}+\gamma \mathbf{P} v
$$

- 이로부터 다음을 얻는다.

$$
v=(\mathbf{I}-\gamma \mathbf{P})^{-1}\mathbf{R}
$$

## Markov Decision Process (Markov Decision Process, MDP)

- MRP는 $\langle S,\, \mathcal{P}_{ss'},\, \mathcal{R}_s,\, \gamma \rangle$ 로 정의된다. 여기에 $s_i$ 상태에서 어떤 행동 $a_j$ 를 취했을 때 $s_{k}$ 로 전이할 확률을 생각 할 수 있다. 이때 모든 사건 집합에서 취할 수 있는 행동(혹은 정책)의 집합을 $A=\{a_1,\, a_2,\ldots,\,a_M\} $ 이라 하자. 

- $\mathcal{P}_{s s'}^a = \mathbb{P}[s_{t+1}|s_t=s,\, a_t=a]$ : 상태변이확률 행렬. 이를 $\mathbf{P}^{a}$ 라 하자.

- $\mathcal{R}_{s}^a = \mathbb{E}[r_{t+1}:s_t=s,\, a_t=a]$ : 보상 행렬. 이를 $\mathbf{R}^a$ 이라 하자.

- $\pi (s,\,a)$ 는 $s\in S$ 상태에서 $a\in A$ action을 취할 확률로 정의된다. 즉 $\mathbb{P}[a_t=a|s_t=s]$ 이다. 이를 policy (정책) 이라 한다. 

- 그렇다면 $s\to s'$ 일 확률을 구해보자. 이를 $\pi$ 라는 policy 를 가정하므로 이를 $\mathcal{P}^\pi_{ss'}$ 이라 하자. 그리고 이에 대한 보상벡터 $\mathcal{R}^{\pi}_s$ 를 생각하면, 

$$
\begin{align}
\mathcal{P}_{ss'}^\pi &= \sum_{a\in A} \pi (s,\,a) \mathcal{P}^a_{ss'} \\
\mathcal{R}_{s}^\pi &=\sum_{a\in A} \pi (s,\,a) \mathcal{R}^a_{s}
\end{align}
$$

이 된다. 이제 이 행렬들을 각각 $\mathbf{P}^{\pi},\, \mathbf{R}^{\pi}$ 라 하자. 


- 행동가치함수 $q_{\pi}(s,\,a)$ 를 $s$ 상태에서 $a$ action을 취했을 때 기대되는 가치로 정의하자. 

$$
q_{\pi}(s,\,a) = R_s^a + \gamma \sum_{s'\in S} \mathcal{P}_{ss'}^a v_\pi (s') \tag{1}
$$

- 이 때 상태가치함수 $v_{\pi}(s)$ 는 다음과 같이 기술 될 수 있을 것이다. 

$$
v_{\pi}(s) = \sum_{a\in A} \pi (s,\,a) q_\pi (s,\,a) \tag{2}
$$

이로부터

$$
\begin{align}
v_{\pi}(s) &= \sum_{a\in A} \pi (s,\,a) \left[ R_s^a + \gamma \sum_{s'\in S} \mathcal{P}_{ss'}^a v_{\pi} (s') \right] \\
&=\sum_{a\in A} \pi (s,\,a) R_s^a + \gamma \sum_{s'\in S} \left( \sum_{a\in A} \pi (s,\,a) \mathcal{P}_{ss'}^a \right)
\end{align}
$$

이므로 

$$
v_{\pi}^{(t)}=\mathbf{R}^\pi + \gamma \mathbf{P}^{\pi}v_{\pi}^{(t+1)}
$$

이다. MRP 에서 처럼 다음과 같이 쓸 수 있다.

$$
v_\pi =\left( \mathbf{I}-\gamma \mathbf{P}^\pi \right)^{-1}\mathbf{R}^\pi\;.
$$